In [1]:
import numpy as np
from pandas import DataFrame

In [2]:
with open('./Pressure_vessels.txt','r') as f:
    data = f.read()

data = data.split(' ')
data = [float(num) for num in data]
data = np.array(data)

### exercise

In [3]:
n = len(data)

Y = data
global Y, n

In [4]:
def f_lambda(lambd, theta):
    
    return n/lambd + np.log(Y).sum() - ((Y/theta)**lambd * np.log(Y/theta)).sum() - n*np.log(theta)

def f_theta(lambd, theta):
    
    return -(n*lambd)/theta + lambd*((Y**lambd) / theta**(lambd+1)).sum()

def f_lambda_2(lambd, theta):
    
    return -n/lambd**2 - ((Y/theta)**lambd * np.log(Y/theta) ** 2).sum()

def f_theta_2(lambd, theta):
    
    return n*lambd/theta**2 - lambd*(lambd+1) * (Y**lambd / theta**(lambd+2)).sum()

def f_cross(lambd, theta):
    
    return (lambd*(Y**lambd / theta**(lambd+1)) * np.log(Y/theta) + Y**lambd/theta**(lambd+1)).sum() -n/theta

global f_lambda, f_theta, f_lambda_2, f_theta_2, f_cross

In [5]:
#define hessian matrix
def Hessian(lambd, theta, frac=0.00001):
    H = np.arange(4).reshape(2,2)
    
    H[0,0] = f_lambda_2(lambd, theta)
    H[0,1] = f_cross(lambd, theta)
    H[1,0] = f_cross(lambd, theta)
    H[1,1] = f_theta_2(lambd, theta)
    
    return H + frac*np.diag([1,1])

# define gradient
def gradient(lambd, theta):
    G = np.arange(2).reshape(2,1)
    
    G[0,0] = f_lambda(lambd, theta)
    G[1,0] = f_theta(lambd, theta)
    
    return G

In [6]:
# define gradient
def gradient(lambd, theta):
    G = np.arange(2).reshape(2,1)
    
    G[0,0] = f_lambda(lambd, theta)
    G[1,0] = f_theta(lambd, theta)
    
    return G

In [7]:
gradient(0.5, 1)

array([[-39591],
       [  2181]])

In [8]:
np.random.randn(2,1)

array([[-0.20336779],
       [-0.2852651 ]])

In [9]:
ta = np.arange(2).reshape(2,1)
np.max(ta)

1

## using newton method

In [62]:
X = np.array([2,9000])
X[1]

9000

In [10]:
#newton method
Y = data
maxiter = 1000
eps = 0.001
k = 0
X = np.array([[1.5],[9000]])

while k < maxiter:
    
    #print(k)
    X_star = X - np.linalg.inv(Hessian(X[0,0], X[1,0])) @ gradient(X[0,0], X[1,0])

    if np.abs(np.max(X_star - X)) < eps:
       break

    X = X_star

    k = k + 1

In [11]:
X

array([[1.88375012e+00],
       [9.00000000e+03]])

In [12]:
print('the maximum point of likelihood function X_star: \n', np.squeeze(X_star))

the maximum point of likelihood function X_star: 
 [1.88375012e+00 9.00000000e+03]


## using bootstrap to estimate interval 

In [13]:
def bootstrap(X):
    X = DataFrame(X)
    sample = X.sample(frac = 1, replace = True)
    return np.squeeze(np.array(sample),1)

In [14]:
para_list = []
n = 0
maxiter = 1000
eps = 0.001

while n < 100:
    print(n)
    Y = bootstrap(data)
    k = 0
    X = np.array([[1.5],[9000]])

    while k < maxiter:

        #print(k)
        X_star = X - np.linalg.inv(Hessian(X[0,0], X[1,0])) @ gradient(X[0,0], X[1,0])

        if np.abs(np.max(X_star - X)) < eps:
            break

        X = X_star

        k = k + 1
    
    para_list.append(np.squeeze(X_star))
    n = n + 1

0


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in power
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in log
  # This is added back by InteractiveShellApp.init_path()


ValueError: cannot convert float NaN to integer

## gradient descent 

In [16]:
Y = data
X = np.array([[1.5],[9000]])
mu = 0.001
maxiter = 1000
eps = 0.001
k = 0

while k < maxiter:
    
    X_star = X - mu * gradient(X[0,0], X[1,0])
    
    if np.max(X_star - X) < eps:
        break
        
    k = k + 1

X_star

array([[1.076e+00],
       [9.000e+03]])